# MUST RUN AT THE START OF EVERYTHING

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])

In [ ]:
from snorkel.viewer import SentenceNgramViewer

candidates = session.query(DiseaseGene).filter(DiseaseGene.split==0).all()
sv = SentenceNgramViewer(candidates, session)

In [ ]:
sv

# Label Functions for the label manager

In [3]:
import re
from snorkel.lf_helpers import (
    get_left_tokens,
    get_right_tokens, 
    get_between_tokens,
    get_tagged_text,
    get_text_between,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

/home/davidnicholson/anaconda3/envs/snorkeling/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
#insert label functions here!!!
#For my hackish debugger purposes only
def LF_debugger(c):
    print c
    print
    print vars(c)
    print
    print vars(c[0])
    print
    print c[0]
    print 
    print c[0].get_span()
    print
    print get_tagged_text(c)
    print
    print get_text_between(c)
    print
    print get_between_tokens(c)
    print
    print get_left_tokens(c)
    print
    print get_right_tokens(c)
    print
    sys.exit(1)
    return 1 if c else 0

def LF_between_tag(c):
    m = re.search("associated with|Disruption",get_text_between(c))
    return 1 if m else 0

def LF_mutation(c):
    m = re.search("mutation", ",".join(get_left_tokens(c)))
    n = re.search("mutation", ",".join(get_right_tokens(c)))
    return 1 if m or n else 0

def LF_check_disease_tag(c):
    disease_name = c[0].get_span()
    if "syndrome" in disease_name:
        if "epilepsy" in disease_name.replace("syndrome",""):
            return 1
        else:
            return -1
    else:
        return 1 if "epilepsy" in disease_name else 0

In [5]:
LFs = [
    #LF_debugger
    LF_between_tag,
    LF_mutation,
    LF_check_disease_tag,
]

# Label the candidates below

In [6]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(f=LFs)

%time L_train = labeler.apply(split=0)
%time L_dev = labeler.apply_existing(split=1)

Clearing existing...
Running UDF...
[========================================] 100%%

CPU times: user 7.66 s, sys: 48 ms, total: 7.71 s
Wall time: 8.01 s
Clearing existing...
Running UDF...
[========================================] 100%%

CPU times: user 7.96 s, sys: 64 ms, total: 8.02 s
Wall time: 8.1 s


# Genreate Coverage Stats

In [7]:
print L_train.lf_stats(session, )
print L_dev.lf_stats(session, )

                      j  Coverage  Overlaps  Conflicts
LF_between_tag        0  0.092243  0.041929   0.026555
LF_mutation           1  0.129979  0.048218   0.022362
LF_check_disease_tag  2  0.306080  0.080363   0.048218
                      j  Coverage  Overlaps  Conflicts
LF_between_tag        0  0.054765  0.026316   0.009957
LF_mutation           1  0.125178  0.046942   0.024182
LF_check_disease_tag  2  0.265292  0.050498   0.031294
